# CrewAI Advanced Level: Flows and State Management

## Mastering Production-Ready AI Workflows

**Flows** are the backbone of CrewAI applications. They provide structured, event-driven workflows with state management and precise control over execution.

### What You'll Learn:
1. Understanding Flows architecture
2. Creating your first Flow
3. State management across steps
4. Event-driven execution with listeners
5. Integrating Crews within Flows
6. Conditional logic and branching

### Prerequisites:
- Completed Entry and Middle Level notebooks
- Understanding of Agents, Tasks, and Crews

---

## Setup

In [1]:
# Install packages
%pip install crewai crewai-tools python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# ============================================
# GLOBAL CONFIGURATION
# ============================================
MODEL = "gpt-5-mini"

# Import CrewAI components
from crewai import Agent, Task, Crew, Process, LLM
from crewai.flow.flow import Flow, listen, start, router
from pydantic import BaseModel
from typing import Optional

# Initialize LLM
llm = LLM(model=f"openai/{MODEL}", temperature=0.7)

print(f"Environment loaded. Using model: {MODEL}")

Environment loaded. Using model: gpt-5-mini


## Part 1: Understanding Flows

### Flow vs Crew: When to Use Which

| Component | Purpose | Use When |
|-----------|---------|----------|
| **Crew** | Team of agents collaborating | Need autonomous problem-solving |
| **Flow** | Structured workflow with state | Need control, persistence, branching |
| **Flow + Crew** | Best of both worlds | Production applications |

### Flow Concepts:
- **State**: Persistent data across steps
- **Steps**: Individual functions in the workflow
- **Listeners**: Trigger steps based on events
- **Router**: Conditional branching logic

## Part 2: Your First Flow

Let's create a simple Flow that processes user input through multiple steps.

In [3]:
# Define a simple Flow
class SimpleFlow(Flow):
    """A basic flow demonstrating step-by-step execution."""
    
    @start()  # This is the entry point
    def get_input(self):
        """Step 1: Process initial input"""
        print("Step 1: Processing input...")
        return "Hello from Step 1"
    
    @listen(get_input)  # Triggered after get_input completes
    def process_data(self, input_data):
        """Step 2: Process the data from step 1"""
        print(f"Step 2: Received '{input_data}'")
        processed = f"Processed: {input_data.upper()}"
        return processed
    
    @listen(process_data)  # Triggered after process_data completes
    def finalize(self, processed_data):
        """Step 3: Finalize and return result"""
        print(f"Step 3: Finalizing with '{processed_data}'")
        return f"Final result: {processed_data}"

# Create and run the flow
simple_flow = SimpleFlow()
result = simple_flow.kickoff()

print("\n" + "="*50)
print("Flow completed!")
print(f"Result: {result}")

RuntimeError: asyncio.run() cannot be called from a running event loop

## Part 3: State Management

Flows can maintain state using Pydantic models. This allows data to persist across steps and even across flow executions.

In [ ]:
# Define state model using Pydantic
class ContentState(BaseModel):
    """State model for content generation workflow"""
    topic: str = ""
    research_results: str = ""
    draft_content: str = ""
    final_content: str = ""
    word_count: int = 0
    status: str = "initialized"

# Flow with state management
class ContentFlow(Flow[ContentState]):
    """Flow that maintains state across steps"""
    
    @start()
    def set_topic(self):
        """Initialize the topic"""
        self.state.topic = "Benefits of AI in Education"
        self.state.status = "topic_set"
        print(f"Topic set: {self.state.topic}")
        return self.state.topic
    
    @listen(set_topic)
    def research_topic(self, topic):
        """Simulate research step"""
        print(f"Researching: {topic}")
        # In real scenario, this could call a Crew
        self.state.research_results = f"Research findings for '{topic}': AI improves personalized learning, automates grading, and provides 24/7 tutoring support."
        self.state.status = "researched"
        return self.state.research_results
    
    @listen(research_topic)
    def create_draft(self, research):
        """Create draft based on research"""
        print("Creating draft...")
        self.state.draft_content = f"Draft Article: {self.state.topic}\n\nBased on our research: {research[:100]}..."
        self.state.status = "drafted"
        return self.state.draft_content
    
    @listen(create_draft)
    def finalize_content(self, draft):
        """Finalize the content"""
        print("Finalizing content...")
        self.state.final_content = f"FINAL: {draft}"
        self.state.word_count = len(self.state.final_content.split())
        self.state.status = "completed"
        return self.state

# Run the stateful flow
content_flow = ContentFlow()
final_state = content_flow.kickoff()

print("\n" + "="*50)
print("Flow State Summary:")
print(f"Status: {content_flow.state.status}")
print(f"Topic: {content_flow.state.topic}")
print(f"Word Count: {content_flow.state.word_count}")

Flow started with ID: 5fa747c8-503f-4f2c-9866-87e2546e8ffa

Topic set: Benefits of AI in Education


╭─────────────────────────────────────────────── 🌊 Flow Execution ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name:                                                                                                          │
│  ContentFlow                                                                                                    │
│  ID:                                                                                                            │
│  5fa747c8-503f-4f2c-9866-87e2546e8ffa                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Researching: Benefits of AI in Education


Creating draft...


...

╭─────────────────────────────────────────── ✅ Flow Method Completed ────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: finalize_content                                                                                       │
│  Status: Completed                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── ✅ Flow Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  ContentFlow                                                                                                    │
│  ID:                                                                                                            │
│  5fa747c8-503f-4f2c-9866-87e2546e8ffa                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🌊 Flow Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Started                                                                                                   │
│  Name: ContentFlow                                                                                              │
│  ID: 5fa747c8-503f-4f2c-9866-87e2546e8ffa                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔄 Flow Method Running ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: create_draft                                                                                           │
│  Status: Running                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Flow State Summary:
Status: completed
Topic: Benefits of AI in Education
Word Count: 26


## Part 4: Integrating Crews within Flows

The real power comes from combining Flows with Crews. The Flow manages the overall workflow, while Crews handle complex AI tasks.

In [ ]:
# Define state for article generation
class ArticleState(BaseModel):
    topic: str = ""
    research: str = ""
    article: str = ""
    
# Flow that uses a Crew
class ArticleGenerationFlow(Flow[ArticleState]):
    """Flow that delegates to a Crew for content generation"""
    
    @start()
    def initialize(self):
        """Set up the topic"""
        self.state.topic = "The Impact of AI on Remote Work"
        print(f"Initialized with topic: {self.state.topic}")
        return self.state.topic
    
    @listen(initialize)
    def research_with_crew(self, topic):
        """Use a Crew to perform research"""
        print(f"Delegating research to Crew...")
        
        # Create research agent
        researcher = Agent(
            role="Research Analyst",
            goal="Provide comprehensive research on the given topic",
            backstory="Expert researcher with deep knowledge across domains",
            llm=llm,
            verbose=True
        )
        
        # Create research task
        research_task = Task(
            description=f"Research the following topic thoroughly: {topic}. Provide key findings, trends, and insights.",
            expected_output="A detailed research summary with key points",
            agent=researcher
        )
        
        # Create and run crew
        research_crew = Crew(
            agents=[researcher],
            tasks=[research_task],
            verbose=True
        )
        
        result = research_crew.kickoff()
        self.state.research = str(result)
        return self.state.research
    
    @listen(research_with_crew)
    def write_with_crew(self, research):
        """Use a Crew to write the article"""
        print("Delegating writing to Crew...")
        
        # Create writer agent
        writer = Agent(
            role="Content Writer",
            goal="Create engaging articles based on research",
            backstory="Professional writer skilled at making complex topics accessible",
            llm=llm,
            verbose=True
        )
        
        # Create writing task
        writing_task = Task(
            description=f"""Write a compelling article based on this research:
            
            Topic: {self.state.topic}
            Research: {research}
            
            Create a well-structured article with introduction, main points, and conclusion.""",
            expected_output="A polished article ready for publication",
            agent=writer
        )
        
        # Create and run crew
        writing_crew = Crew(
            agents=[writer],
            tasks=[writing_task],
            verbose=True
        )
        
        result = writing_crew.kickoff()
        self.state.article = str(result)
        return self.state.article

print("ArticleGenerationFlow defined!")

ArticleGenerationFlow defined!


In [ ]:
# Run the Flow with Crew integration
# Note: This will make actual API calls

article_flow = ArticleGenerationFlow()
result = article_flow.kickoff()

print("\n" + "="*60)
print("ARTICLE GENERATION COMPLETE!")
print("="*60)
print(f"\nTopic: {article_flow.state.topic}")
print(f"\nFinal Article:\n{article_flow.state.article}")

Flow started with ID: a8611a78-5c58-43b8-ad09-86ce5fbde176

╭─────────────────────────────────────────────── 🌊 Flow Execution ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name:                                                                                                          │
│  ArticleGenerationFlow                                                                                          │
│  ID:                                                                                                            │
│  a8611a78-5c58-43b8-ad09-86ce5fbde176                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Initialized with topic: The Impact of AI on Remote Work
Delegating research to Crew...


╭──────────────────────────────────────────── 🔄 Flow Method Running ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: initialize                                                                                             │
│  Status: Running                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── ✅ Flow Method Completed ────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: initialize                                                                                             │
│  Status: Completed                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  5f341c9d-9931-4b4d-9e47-cd121098b984                                                                           │
│  Final Output: # The Impact of AI on Remote Work: Transforming the Future of Work                               │
│                                                                                                                 │
│  In recent years, the rise of Artificial Intelligence (AI) has ushered in a new era across various sectors,     │
│  and the realm of remote work is no exception. As organizations increasingly embrace remote work models, AI     │
│  technologies have emerged as powerful tools that enhance productivity, improve communication, and personalize  │
│  the employee experience. This article explores the profound impact of AI on remote work, delving into the      │
│  efficiencies it brings, the challenges it poses, and the future trends that may shape this evolving            │
│  landscape.                                                                                                     │
│                                                                                                                 │
│  ## Enhanced Productivity and Efficiency                                                                        │
│                                                                                                                 │
│  One of the most significant contributions of AI to remote work is its ability to boost productivity and        │
│  streamline efficiency. By automating repetitive tasks, AI allows employees to dedicate their time and energy   │
│  to higher-order thinking and creative problem-solving.                                                         │
│                                                                                                                 │
│  **Task Automation** is a prime example. AI-powered project management tools can handle scheduling, reminders,  │
│  and follow-ups automatically, drastically reducing the hours spent on administrative duties. This automation   │
│  frees remote workers to focus on more strategic tasks that require human insight and creativity.               │
│                                                                                                                 │
│  Moreover, AI excels in **Data Analysis**. It can sift through vast datasets quickly, providing insights that   │
│  would take humans much longer to derive. For teams working with complex data, AI’s rapid analysis              │
│  capabilities can lead to quicker decision-making and enhanced performance.                                     │
│                                                                                                                 │
│  ## Improved Communication and Collaboration                                                                    │
│                                                                                                                 │
│  Effective communication is vital in remote work environments, and AI technologies play a crucial role in       │
│  enhancing collaboration among team members.                                                                    │
│                                                       

╭─────────────────────────────────────────── ✅ Flow Method Completed ────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: write_with_crew                                                                                        │
│  Status: Completed                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── ✅ Flow Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  ArticleGenerationFlow                                                                                          │
│  ID:                                                                                                            │
│  a8611a78-5c58-43b8-ad09-86ce5fbde176                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


ARTICLE GENERATION COMPLETE!

Topic: The Impact of AI on Remote Work

Final Article:
# The Impact of AI on Remote Work: Transforming the Future of Work

In recent years, the rise of Artificial Intelligence (AI) has ushered in a new era across various sectors, and the realm of remote work is no exception. As organizations increasingly embrace remote work models, AI technologies have emerged as powerful tools that enhance productivity, improve communication, and personalize the employee experience. This article explores the profound impact of AI on remote work, delving into the efficiencies it brings, the challenges it poses, and the future trends that may shape this evolving landscape.

## Enhanced Productivity and Efficiency

One of the most significant contributions of AI to remote work is its ability to boost productivity and streamline efficiency. By automating repetitive tasks, AI allows employees to dedicate their time and energy to higher-order thinking and creative problem-solv

## Part 5: Conditional Routing

Use the `@router` decorator to create conditional branches in your Flow.

In [4]:
# Flow with conditional routing
class ContentTypeState(BaseModel):
    content_type: str = ""
    content: str = ""
    result: str = ""

class ConditionalFlow(Flow[ContentTypeState]):
    """Flow demonstrating conditional routing based on content type"""
    
    @start()
    def classify_content(self):
        """Classify the content type"""
        # Simulate content classification
        # In real scenario, this could be determined by user input or AI
        self.state.content_type = "technical"  # Options: "technical", "marketing", "casual"
        self.state.content = "AI and Machine Learning"
        print(f"Content classified as: {self.state.content_type}")
        return self.state.content_type
    
    @router(classify_content)
    def route_to_writer(self, content_type):
        """Route to appropriate writer based on content type"""
        if content_type == "technical":
            return "technical_path"
        elif content_type == "marketing":
            return "marketing_path"
        else:
            return "casual_path"
    
    @listen("technical_path")
    def write_technical(self):
        """Handle technical content"""
        print("Routing to technical writer...")
        self.state.result = f"TECHNICAL: Detailed analysis of {self.state.content}"
        return self.state.result
    
    @listen("marketing_path")
    def write_marketing(self):
        """Handle marketing content"""
        print("Routing to marketing writer...")
        self.state.result = f"MARKETING: Compelling copy about {self.state.content}"
        return self.state.result
    
    @listen("casual_path")
    def write_casual(self):
        """Handle casual content"""
        print("Routing to casual writer...")
        self.state.result = f"CASUAL: Fun take on {self.state.content}"
        return self.state.result

# Run the conditional flow
conditional_flow = ConditionalFlow()
result = await conditional_flow.kickoff_async()

print("\n" + "="*50)
print(f"Route taken: {conditional_flow.state.content_type}_path")
print(f"Result: {conditional_flow.state.result}")

Flow started with ID: 6b50d7bc-48ad-48a3-918f-465d1f6696e5

╭─────────────────────────────────────────────── 🌊 Flow Execution ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name:                                                                                                          │
│  ConditionalFlow                                                                                                │
│  ID:                                                                                                            │
│  6b50d7bc-48ad-48a3-918f-465d1f6696e5                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Content classified as: technical


╭──────────────────────────────────────────── 🔄 Flow Method Running ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: classify_content                                                                                       │
│  Status: Running                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Routing to technical writer...


...

╭─────────────────────────────────────────── ✅ Flow Method Completed ────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: route_to_writer                                                                                        │
│  Status: Completed                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔄 Flow Method Running ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: write_technical                                                                                        │
│  Status: Running                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── ✅ Flow Method Completed ────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: write_technical                                                                                        │
│  Status: Completed                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── ✅ Flow Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  ConditionalFlow                                                                                                │
│  ID:                                                                                                            │
│  6b50d7bc-48ad-48a3-918f-465d1f6696e5                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Route taken: technical_path
Result: TECHNICAL: Detailed analysis of AI and Machine Learning


## Part 6: Passing Inputs to Flows

You can pass external inputs to a Flow when kicking it off.

In [7]:
# Flow that accepts external inputs
class InputState(BaseModel):
    user_query: str = ""
    processed_query: str = ""
    response: str = ""

class QueryFlow(Flow[InputState]):
    """Flow that processes user queries"""
    
    @start()
    def receive_query(self):
        """Receive and validate the query"""
        query = self.state.user_query
        print(f"Received query: {query}")
        return query
    
    @listen(receive_query)
    def process_query(self, query):
        """Process the query"""
        self.state.processed_query = query.strip().lower()
        print(f"Processed query: {self.state.processed_query}")
        return self.state.processed_query
    
    @listen(process_query)
    def generate_response(self, processed):
        """Generate a response"""
        self.state.response = f"Response to '{processed}': This is a simulated answer."
        return self.state.response

# Run with input
query_flow = QueryFlow()

# Method 1: Set state before kickoff
query_flow.state.user_query = "What is machine learning?"
result1 = query_flow.kickoff()
print(f"\nResult 1: {query_flow.state.response}")

# Method 2: Pass inputs directly
query_flow2 = QueryFlow()
result2 = query_flow2.kickoff(inputs={"user_query": "How does AI work?"})
print(f"\nResult 2: {query_flow2.state.response}")

Flow started with ID: 952230ab-bf68-44df-9116-4969afb8d0a6

Received query: What is machine learning?


╭─────────────────────────────────────────────── 🌊 Flow Execution ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name:                                                                                                          │
│  QueryFlow                                                                                                      │
│  ID:                                                                                                            │
│  952230ab-bf68-44df-9116-4969afb8d0a6                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Processed query: what is machine learning?


╭──────────────────────────────────────────── 🔄 Flow Method Running ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: receive_query                                                                                          │
│  Status: Running                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔄 Flow Method Running ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: generate_response                                                                                      │
│  Status: Running                                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── ✅ Flow Method Completed ────────────────────────────────────────────╮
│                                                                                                                 │
│  Method: generate_response                                                                                      │
│  Status: Completed                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── ✅ Flow Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  QueryFlow                                                                                                      │
│  ID:                                                                                                            │
│  4d9392c9-9d47-48a4-8cf4-a23d854a9043                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🌊 Flow Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Started                                                                                                   │
│  Name: QueryFlow                                                                                                │
│  ID: 4d9392c9-9d47-48a4-8cf4-a23d854a9043                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Result 2: Response to 'how does ai work?': This is a simulated answer.


In [ ]:
# EXERCISE: Customer Service Flow

class CustomerServiceState(BaseModel):
    inquiry: str = ""
    category: str = ""  # "billing", "technical", "general"
    response: str = ""

class CustomerServiceFlow(Flow[CustomerServiceState]):
    """Your customer service flow"""
    
    @start()
    def receive_inquiry(self):
        # TODO: Implement
        pass
    
    @router(receive_inquiry)
    def categorize_inquiry(self, inquiry):
        # TODO: Implement routing logic
        # Return "billing_path", "technical_path", or "general_path"
        pass
    
    @listen("billing_path")
    def handle_billing(self):
        # TODO: Create billing support Crew
        pass
    
    @listen("technical_path")
    def handle_technical(self):
        # TODO: Create technical support Crew
        pass
    
    @listen("general_path")
    def handle_general(self):
        # TODO: Create general support Crew
        pass

# Test your flow
# cs_flow = CustomerServiceFlow()
# cs_flow.state.inquiry = "I have a question about my bill"
# result = cs_flow.kickoff()
# print(result)

## Summary

In this notebook, you mastered:
- Creating Flows with start and listen decorators
- State management with Pydantic models
- Integrating Crews within Flows
- Conditional routing with the router decorator
- Passing inputs to Flows

Key decorators:
- @start() - Entry point of the Flow
- @listen(step) - Triggered after another step
- @router(step) - Conditional branching

Next Steps: Move to the Specialty Level notebook for custom tools and production patterns